# Pseudo dataset
* Get sample data [here](https://github.com/fuyu-quant/IBLM/tree/main/datasets).

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@feature-in-context-learning 

In [2]:
#%%capture
#!pip install git+https://github.com/fuyu-quant/IBLM.git

In [3]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.39


### Training

In [4]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier

import sys
import os

In [5]:
columns = 4
train_data = 300

file_name = f'pseudodata_{columns}f_{train_data}_train.csv'


df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_train = df.drop('target', axis=1)
y_train = df['target']

In [6]:

llm_model = OpenAI(temperature=0, model_name = 'gpt-4')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model_name = 1, llm_model = llm_model, params=params)

In [7]:
file_path = '../models/pseudodata/'

model = iblm.fit(x_train, y_train, model_name = 'pseudodata', file_path=file_path)

> Start of model creating.
Tokens Used: 6585
	Prompt Tokens: 6437
	Completion Tokens: 148
Successful Requests: 1
Total Cost (USD): $0.20198999999999998


In [8]:
# Code of the model created
print(model)

import numpy as np

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        
        # Calculate the weighted sum of the input features
        weighted_sum = row['a'] * 0.3 + row['b'] * 0.2 + row['c'] * 0.4 + row['d'] * 0.1
        
        # Apply the sigmoid function to the weighted sum to get the probability
        y = 1 / (1 + np.exp(-weighted_sum))

        # Do not change the code after this point.
        output.append(y)
    return np.array(output)


### Prediction

In [9]:
file_name = f'pseudodata_{columns}f_{train_data}_test.csv'


df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_test = df.drop('target', axis=1)
y_test = df['target']

In [10]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [11]:
accuracy = round(accuracy_score(y_test, y_pred),4)
print(f'Accuracy: {accuracy}')

# Precision
precision = round(precision_score(y_test, y_pred),4)
print(f'Precision: {precision}')

# Recall
recall = round(recall_score(y_test, y_pred),4)
print(f'Recall: {recall}')

# F1 score
f1 = round(f1_score(y_test, y_pred),4)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = round(roc_auc_score(y_test, y_proba),4)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.4829
Precision: 0.4811
Recall: 0.4371
F1 score: 0.4581
ROC-AUC: 0.553


### Creating Multiple Models

In [12]:
columns = 4
train_data = 300

file_name = f'pseudodata_{columns}f_{train_data}_train.csv'


df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_train = df.drop('target', axis=1)
y_train = df['target']

file_path = '../models/pseudodata/'
for i in range(1,31):
    model = iblm.fit(x_train, y_train, model_name = f'pseudodata_{columns}f_{train_data}_{i}', file_path=file_path)

> Start of model creating.
Tokens Used: 6585
	Prompt Tokens: 6437
	Completion Tokens: 148
Successful Requests: 1
Total Cost (USD): $0.20198999999999998
> Start of model creating.
Tokens Used: 6585
	Prompt Tokens: 6437
	Completion Tokens: 148
Successful Requests: 1
Total Cost (USD): $0.20198999999999998
> Start of model creating.
Tokens Used: 6585
	Prompt Tokens: 6437
	Completion Tokens: 148
Successful Requests: 1
Total Cost (USD): $0.20198999999999998
> Start of model creating.
Tokens Used: 6585
	Prompt Tokens: 6437
	Completion Tokens: 148
Successful Requests: 1
Total Cost (USD): $0.20198999999999998
> Start of model creating.
Tokens Used: 6585
	Prompt Tokens: 6437
	Completion Tokens: 148
Successful Requests: 1
Total Cost (USD): $0.20198999999999998
> Start of model creating.
Tokens Used: 6585
	Prompt Tokens: 6437
	Completion Tokens: 148
Successful Requests: 1
Total Cost (USD): $0.20198999999999998
> Start of model creating.
Tokens Used: 6585
	Prompt Tokens: 6437
	Completion Tokens: 14

### Prediction

In [14]:
columns = 4
train_data = 300

file_name = f'pseudodata_{columns}f_{train_data}_test.csv'


df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_test = df.drop('target', axis=1)
y_test = df['target']

sys.path.append('..')

error_count = 0
error_list = []
auc_list = []


for i in range(1,31):

    import_file = f'import models.pseudodata.pseudodata_{columns}f_{train_data}_{i} as codemodel'

    exec(import_file)

    try:
        y_proba = codemodel.predict(x_test)
        y_pred = (y_proba > 0.5).astype(int)
        negative_values_exist = np.any(y_proba < 0)
        values_greater_than_one_exist = np.any(y_proba > 1)
        if negative_values_exist:
            error_list.append(i)
            error_count += 1
            print(f"Negative values exist：{negative_values_exist}")

        elif values_greater_than_one_exist:
            error_list.append(i)
            error_count += 1
            print(f"Positive values exist：{values_greater_than_one_exist}")

        else:
            roc_auc = roc_auc_score(y_test, y_proba)
            auc_list.append(roc_auc)
        
    except Exception:
        print('run error')
        error_count += 1
        error_list.append(i)
        pass

print(error_count)
print(error_list)
print(auc_list)
average = sum(auc_list) / len(auc_list)
print("Average Value:", average)
max_value = max(auc_list)
min_value = min(auc_list)
print("Maximum Value:", max_value)
print("Minimum Value:", min_value)

run error
run error
2
[25, 30]
[0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755, 0.5529673469387755]
Average Value: 0.5529673469387754
Maximum Value: 0.5529673469387755
Minimum Value: 0.5529673469387755
